Run <b>3 cells below</b> and then <b>after <i>Up-Scale and get perfomance of your models</i></b>

In [11]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

import sys
import os 
from datetime import datetime
import random as rnd
import itertools as it
#--------------------------------------------------------------------------------------------------------------------------
from RECOGNIZER.recognizer import Recognizer
from skimage.io import imread
#--------------------------------------------------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_basename(path): # dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco_0001.JPG -> Abel_Pacheco_0001
    return os.path.splitext(os.path.basename(path))[0]

def add_pathes(df, scales, model_name): # (df, [2, 3, 4], 'DCSCN')
    for scale in scales:
        df[f'path_x{scale}'] = f'{model_name}/output_x{scale}/' + df[f'path_x1'].apply(get_basename) + '_result.jpg'
    return df

def get_pathes(classification_df, df):
    for pic_num in [1, 2]:
        classification_df = (pd.merge(classification_df, df, 
                                      left_on=[f'name_{pic_num}', f'pic_{pic_num}'], right_on=['name', 'pic'], 
                                      how='left', suffixes=('_1', '_2')))
    classification_df = classification_df.loc[:, ~classification_df.columns.duplicated()]
    return classification_df

In [3]:
def recognize_pics(classification_df, scales):
    initial_time = datetime.now() 
    print(f'Started at {initial_time}')

    recognizer = Recognizer()
    THRESHOLD = 0.9
    
    def recongize(row):
        photo1 = imread(row.ix[0])
        photo2 = imread(row.ix[1])
        similarity = recognizer.get_best_similarity(photo1, photo2)
        return 1 if similarity < THRESHOLD else 0

    for scale in scales:
        start = datetime.now()
        print(f'Scale x{scale} started at {start}')
        classification_df[f'rec_x{scale}'] = classification_df[[f'path_x{scale}_1', f'path_x{scale}_2']].apply(recongize, axis=1)
        print(f'Scale x{scale} is done in {datetime.now() - start}')
    print(f'Ended in {datetime.now() - initial_time}')
    return classification_df

# Creating a DataFrame with paths to unique pictures

In [ ]:
dataset_path = 'dataset/'

pairs = ''
with open(f'{dataset_path}pairs.txt', 'r') as file:
    pairs = file.read()

df = pd.DataFrame(list(map(lambda x: x.split('\t'), pairs.split('\n')[1:301])), columns=['name', 'pic1', 'pic2'])
df = (pd.melt(df, id_vars='name', value_vars=['pic1', 'pic2'], var_name='picNum', value_name='pic')
        .drop('picNum', axis=1)
        .sort_values(by=['name', 'pic'])
        .drop_duplicates()
        .reset_index(drop=True))

df['pic'] = df['pic'].apply(lambda x: x.zfill(4))
df['path_x1'] = str(dataset_path) + df['name'] + '/' + df['name'] + '_' + df['pic'] + '.jpg'

df.to_csv('pics_df.csv')
df.head()

In [ ]:
pics_to_scale_paths = df['path_x1'].values
pics_to_scale = list(map(lambda x: get_basename(x) + '_result.jpg', pics_to_scale_paths))
print(f'There are {len(pics_to_scale)} unique pics to be scaled')

# Creating regular DataFrame for classification

In [ ]:
sim_people_pics = []
for name, name_df in df[['name', 'pic']].groupby('name'):
    pics = name_df['pic'].values
    pics_comb = list(it.combinations(pics, 2))
    pics_name_comb = list(map(lambda x: [name]+[x[0]]+[name]+[x[1]], pics_comb))
    for comb in pics_name_comb:
        sim_people_pics.append(comb+[1])

non_sim_people_pics = [el for el in it.combinations(df[['name', 'pic']].values.tolist(), 2) if el[0][0] != el[1][0]]
non_sim_people_pics = [el[0]+el[1]+[0] for el in non_sim_people_pics]
non_sim_people_pics = rnd.sample(non_sim_people_pics, len(sim_people_pics))

classification_df = pd.DataFrame(sim_people_pics+non_sim_people_pics, 
                                 columns=['name_1', 'pic_1', 'name_2', 'pic_2', 'similarity'])

classification_df.to_csv('classification_df.csv')
classification_df.iloc[[0, 1, 2, -3, -2, -1], :]

In [ ]:
classification_df = get_pathes(classification_df, df)
classification_df.iloc[[0, 1, 2, -3, -2, -1], :]

In [ ]:
classification_df = recognize_pics(classification_df, [1])

In [ ]:
classification_df.to_csv('classification_df.csv')
classification_df.iloc[[0, 1, 2, -3, -2, -1], :]

# Up-Scale and get perfomance of your models

### Up-Scale funcs of different models

Now you can up-scale pics with different models and look at it's perfomance in recognition

In [ ]:
pics_to_scale_paths = pd.read_csv('pics_df.csv', usecols=['path_x1'])['path_x1'].values
pics_to_scale = list(map(lambda x: get_basename(x) + '_result.jpg', pics_to_scale_paths))
print(f'There are {len(pics_to_scale)} unique pics to be scaled')

In [ ]:
# Модель DCSCN 
# Тут надо самому ставить нужный скейл: 2 3 или 4, так как памяти не хватает + настроить сохранение
# в нужную папку сложна (не хочется заморачиваться), то надо перезапускать и руками переименовывать папку.

model_name = 'DCSCN'
scales = 4
scaled_pics = os.listdir(f'{model_name}/output') # папка, в которой должны находиться апскейленные пикчи
for pic, pic_path in zip(pics_to_scale, pics_to_scale_paths):
    if pic in scaled_pics: # зачастую случается OOM и чтобы заново не апскейлить уже апскейленные пикчи, чекаем их
        continue
    abc = f'--file={pic_path} --scale={scale}'
    %run sr.py {abc}

In [ ]:
# Модель ESRGAN
import glob
import cv2
import torch
import ESRGAN.architecture as arch

model_path = 'ESRGAN/models/RRDB_ESRGAN_x4.pth' # sys.argv[1]  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth
device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu

model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=4, norm_type=None, act_type='leakyrelu', 
                      mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

scale = 4
for pic, pic_path in zip(pics_to_scale, pics_to_scale_paths):
    img = cv2.imread(pic_path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite(f'dataset/output_ESRGAN_x4/{pic}', output)

In [ ]:
# YOUR UP-SCALE FUNCS

In [ ]:
# YOUR UP-SCALE FUNCS

In [ ]:
# YOUR UP-SCALE FUNCS

### Get recognition perfomance of your models

In [4]:
def get_rec_perfomance(model_name, scales):
    pics_df = pd.read_csv('pics_df.csv', index_col=0, dtype='object')
    classification_df = pd.read_csv('classification_df.csv', index_col=0, dtype='object')
    
    model_pics_df = add_pathes(pics_df, scales, model_name)
    model_classification_df = get_pathes(classification_df, model_pics_df)
    
    model_classification_df = recognize_pics(model_classification_df, scales)
    model_classification_df.to_csv(f'classification_df_{model_name}.csv')
    return model_classification_df

In [8]:
models = [['No Scale',  [1]],
          ['DCSCN',     [2, 3, 4]], 
          ['ESRGAN',    [4]],
          ['EDSR',      [2, 3, 4]],
          ['MDSR',      [2]],
          ['SRFBN',     [2, 3, 4]],
          ['SRFBN_BD',  [3]],
          ['SRFBN_DN',  [3]],
          ['DDSRCNN',   [2]],
          ['DBPN',      [2, 4]],
          ['proSR',     [2, 4]],
          ['proSRGAN',  [4]],
          ['proSRs',    [2, 4]],
          ['DBPNLL',    [4]]
         ]

In [ ]:
model_name, scales = models[0] # No Scale
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[1] # DCSCN
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[2] # ESRGAN
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[3] # EDSR
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[4] # MDSR
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[5] # SRFBN
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[6] # SRFBN_BD
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[7] # SRFBN_DN
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[8] # DDSRCNN
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[9] # DBPN
model_df = get_rec_perfomance(model_name, scales)

In [9]:
model_name, scales = models[10] # proSR
model_df = get_rec_perfomance(model_name, scales)

Started at 2019-06-05 22:00:22.012289
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from RECOGNIZER/facenet_models/facenet/model-20180408-102900.ckpt-90
Scale x2 started at 2019-06-05 22:00:48.458599
Scale x2 is done in 0:17:23.770895
Scale x4 started at 2019-06-05 22:18:12.229494
Scale x4 is done in 0:52:31.309183
Ended in 1:10:21.526388


In [10]:
model_name, scales = models[11] # proSRGAN
model_df = get_rec_perfomance(model_name, scales)

Started at 2019-06-05 23:10:43.698881
INFO:tensorflow:Restoring parameters from RECOGNIZER/facenet_models/facenet/model-20180408-102900.ckpt-90
Scale x4 started at 2019-06-05 23:11:06.769503
Scale x4 is done in 0:51:16.752948
Ended in 0:51:39.823570


In [ ]:
model_name, scales = models[12] # proSRs
model_df = get_rec_perfomance(model_name, scales)

In [ ]:
model_name, scales = models[13] # DBPNLL
model_df = get_rec_perfomance(model_name, scales)

### Get the required metrics

In [16]:
metrics_df = pd.DataFrame()

for model_name, scales in models:
    model_df = pd.read_csv(f'classification_df_{model_name}.csv', 
                           usecols=['similarity'] + [f'rec_x{scale}' for scale in scales])
    rec_cols = model_df.columns[1:]
    
    model_accuracy = [round(accuracy_score(model_df['similarity'], model_df[col]) * 100, 2) for col in rec_cols]
    model_precision = [round(precision_score(model_df['similarity'], model_df[col]) * 100, 2) for col in rec_cols]
    model_recall = [round(recall_score(model_df['similarity'], model_df[col]) * 100, 2) for col in rec_cols]
    model_f1 = [round(f1_score(model_df['similarity'], model_df[col]) * 100, 2) for col in rec_cols]
    tn = [round(confusion_matrix(model_df['similarity'], model_df[col]).ravel()[0] / model_df.shape[0] * 200, 2) for col in rec_cols]
    fp = [round(confusion_matrix(model_df['similarity'], model_df[col]).ravel()[1] / model_df.shape[0] * 200, 2) for col in rec_cols]
    fn = [round(confusion_matrix(model_df['similarity'], model_df[col]).ravel()[2] / model_df.shape[0] * 200, 2) for col in rec_cols]
    tp = [round(confusion_matrix(model_df['similarity'], model_df[col]).ravel()[3] / model_df.shape[0] * 200, 2) for col in rec_cols]

    data_to_append = zip([model_name]*len(scales), scales, 
                         model_accuracy, model_precision, model_recall, model_f1, tp, fp, fn, tn)
    metrics_df = metrics_df.append(list(data_to_append))
metrics_df.reset_index(drop=True, inplace=True)
metrics_df.columns = ['Model Name', 'Scale', 'Accuracy', 'Precision', 'Recall', 'F1', 
                      'True Pos', 'False Pos', 'False Neg', 'True Neg']
metrics_df['Rank'] = sum([metrics_df[col].rank(method='first') for col in ['Accuracy', 'Precision', 'Recall']])
metrics_df.to_excel('benchmark.xlsx')
metrics_df.sort_values(by='Rank', ascending=False)
# metrics_df

,Model Name,Scale,Accuracy,Precision,Recall,F1,True Pos,False Pos,False Neg,True Neg,Rank
8,MDSR,2,74.22,70.65,82.85,76.27,82.85,34.41,17.15,65.59,53.0
1,DCSCN,2,73.98,70.12,83.57,76.26,83.57,35.61,16.43,64.39,53.0
22,DBPNLL,4,72.78,68.45,84.53,75.64,84.53,38.97,15.47,61.03,51.0
0,No Scale,1,73.68,70.72,80.82,75.43,80.82,33.45,19.18,66.55,45.0
20,proSRs,2,73.20,69.72,82.01,75.37,82.01,35.61,17.99,64.39,44.0
14,DDSRCNN,2,73.56,70.20,81.89,75.59,81.89,34.77,18.11,65.23,44.0
17,proSR,2,72.84,69.26,82.13,75.15,82.13,36.45,17.87,63.55,43.0
7,EDSR,4,72.36,68.09,84.17,75.28,84.17,39.45,15.83,60.55,41.0
2,DCSCN,3,72.48,68.40,83.57,75.23,83.57,38.61,16.43,61.39,41.0
16,DBPN,4,72.18,67.79,84.53,75.24,84.53,40.17,15.47,59.83,38.0
